In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# read in csv as data sequence
data = pd.read_csv('meteo0.csv', header=None).values

In [3]:
# split string and convert elements to integers keeping the same structure
# V = (v^1, ..., v^N) where v^i = (v^i_1, ..., v^i_T)
V = np.array([np.array([int(i) for i in data[j][0].split()]) for j in range(len(data))])

In [4]:
len(V)

500

In [5]:
# parameters of HMM - transition matrix, emission matrix, initial state distribution
# initial parameter distibrutions are randomly generated and normalised
def random_initial_parameters(n_states=3, n_features=3):

    # transition matrix - a_ij = p(h_t = j|h_t-1 = i)
    A = np.random.rand(n_states, n_states)
    A /= np.sum(A, axis=1, keepdims=True)
    # emission matrix - b_ij = p(v_t = j|h_t = i)
    B = np.random.rand(n_states, n_features)
    B /= np.sum(B, axis=1, keepdims=True)
    # initial state distribution - pi_i = p(h_1 = i)
    pi = np.random.rand(1, n_states)
    pi /= np.sum(pi)

    return A, B, pi

In [6]:
# parameters of HMM - transition matrix, emission matrix, initial state distribution
# initial parameter distibrutions are uniform
def uniform_initial_parameters(n_states=3, n_features=3):

    # transition matrix - a_ij = p(h_t = j|h_t-1 = i)
    A = np.ones((n_states, n_states)) / n_states
    # emission matrix - b_ij = p(v_t = j|h_t = i)
    B = np.ones((n_states, n_features)) / n_features
    # initial state distribution - pi_i = p(h_1 = i)
    pi = np.ones((1, n_states)) / n_states

    return A, B, pi

In [7]:
# parameters of HMM - transition matrix, emission matrix, initial state distribution
# initial parameter distibrutions are generated using k-means clustering
def kmeans_initial_parameters(n_states=3, n_features=3):

    # transition matrix - a_ij = p(h_t = j|h_t-1 = i)
    A = np.zeros((n_states, n_states))
    # emission matrix - b_ij = p(v_t = j|h_t = i)
    B = np.zeros((n_states, n_features))
    # initial state distribution - pi_i = p(h_1 = i)
    pi = np.zeros((1, n_states))

    # k-means clustering
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=n_states, random_state=0).fit(V)
    labels = kmeans.labels_

    # initial state distribution
    for i in range(len(labels)):
        pi[0][labels[i]] += 1
    pi /= np.sum(pi)

    # emission matrix
    for i in range(len(labels)):
        B[labels[i]][V[i][0]] += 1
    B /= np.sum(B, axis=1, keepdims=True)

    # transition matrix
    for i in range(len(labels)-1):
        A[labels[i]][labels[i+1]] += 1
    A /= np.sum(A, axis=1, keepdims=True)

    return A, B, pi

In [8]:
# parameters of HMM - transition matrix, emission matrix, initial state distribution
# initial parameter distibrutions are generated using gaussian mixture model clustering
def gmm_initial_parameters(n_states=3, n_features=3):

    # transition matrix - a_ij = p(h_t = j|h_t-1 = i)
    A = np.zeros((n_states, n_states))
    # emission matrix - b_ij = p(v_t = j|h_t = i)
    B = np.zeros((n_states, n_features))
    # initial state distribution - pi_i = p(h_1 = i)
    pi = np.zeros((1, n_states))

    # gmm clustering
    from sklearn.mixture import GaussianMixture
    gmm = GaussianMixture(n_components=n_states, random_state=0).fit(V)
    labels = gmm.predict(V)

    # initial state distribution
    for i in range(len(labels)):
        pi[0][labels[i]] += 1
    pi /= np.sum(pi)

    # emission matrix
    for i in range(len(labels)):
        B[labels[i]][V[i][0]] += 1
    B /= np.sum(B, axis=1, keepdims=True)

    # transition matrix
    for i in range(len(labels)-1):
        A[labels[i]][labels[i+1]] += 1
    A /= np.sum(A, axis=1, keepdims=True)

    return A, B, pi

In [9]:
# alpha recursion - p(h_{t},v_{1:t}) = alpha_{t+1} = alpha_{t} * A * B
def alpha_recursion(observations, A, B, pi):
    n_states = A.shape[0]
    n_samples = len(observations)
    alpha = np.zeros((n_samples, n_states))
    alpha[0,:] = pi * B[:, observations[0]]

    for t in range(1, n_samples):
        for i in range(n_states):
            alpha[t, i] = (alpha[t-1,:]@A[:,i]) * B[i, observations[t]]

    return alpha

In [10]:
# beta recursion - p(v_{1:t} | h_{t}) = beta_{t} = A * B * beta_{t+1}
def beta_recursion(observations, A, B):
    
    n_states = A.shape[0]
    n_samples = len(observations)
    beta = np.zeros((n_samples, n_states))
    beta[-1] = 1

    for t in range(n_samples-2, -1, -1):
        for i in range(n_states):
            beta[t, i] = (beta[t+1,:]*B[:, observations[t+1]]) @ A[i,:]

    return beta

In [11]:
# single sequence update of A, B
def update(observations, A, B, pi):

    # alpha = p(h_{t},v_{1:t}) - dimensions (T, n_states), beta = p(v_{t+1:T}|h_{t}) - dimensions (T, n_states)
    alpha = alpha_recursion(observations, A, B, pi)
    beta = beta_recursion(observations, A, B)
    
    T = len(observations)

    # xi = p(h_{t}, h_{t+1}, v_{1:T}) - dimensions (n_states, n_states, T-1)
    xi = np.zeros((A.shape[0], A.shape[1], observations.shape[0]-1), )
    for t in range(T-1):
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                # xi_{i,j} = p(h_{t}=i, h_{t+1}=j, v_{1:T})
                xi[i, j, t] = alpha[t, i] * A[i, j] * B[j, observations[t + 1]] * beta[t + 1, j]
        xi[:, :, t] /= np.sum(xi[:, :, t])

    # gamma = p(h_{t}, v_{1:T}) - dimensions (n_states, T)
    gamma = np.zeros((A.shape[0], observations.shape[0]))
    for t in range(T):
        for i in range(A.shape[0]):
            gamma[i, t] = alpha[t, i] * beta[t, i]
        gamma[:, t] /= np.sum(gamma[:, t])

    # update A - sum xi over T and normalise
    A = np.sum(xi, axis=2) / np.sum(np.sum(xi, axis=1), axis=1).reshape((-1, 1))
    
    # update B - sum gamma over T and normalise
    B = np.copy(B)
    denominator = np.sum(gamma, axis=1)
    for k in range(B.shape[1]):
        B[:,k] = np.sum(gamma[:,observations==k], axis=1)

    B /= denominator.reshape((-1, 1))

    # update pi
    pi = gamma[:, 0]

    return A, B, pi

In [12]:
# multiple sequence update of A, B
def update_multi_sequences(observations_list, A, B, pi):
    # Initialize accumulators for A and B updates
    A_accumulator = np.zeros_like(A)
    B_accumulator = np.zeros_like(B)
    pi_accumulator = np.zeros_like(pi)

    # Iterate over observation sequences
    for observations in observations_list:
        # alpha = p(h_{t},v_{1:t}) - dimensions (T, n_states), beta = p(v_{t+1:T}|h_{t}) - dimensions (T, n_states)
        alpha = alpha_recursion(observations, A, B, pi)
        beta = beta_recursion(observations, A, B)
        
        # xi = p(h_{t}, h_{t+1}, v_{1:T}) - dimensions (n_states, n_states, T-1)
        # Compute denominator and numerator
        T = len(observations) - 1
        numerator = alpha[:T, :, None] * A[None, :, :] * B[:, observations[1:]].T[:, None] * beta[1:, None, :]
        denominator = np.sum(numerator, axis=(1,2)).reshape((-1, 1, 1))
        xi = numerator / denominator
    
    gamma = np.sum(xi, axis=2)
    gamma = np.vstack((gamma, np.sum(xi[-1, :, :], axis=1)))
    print(gamma.shape)

    # Accumulate updates
    pi_accumulator += np.sum(gamma, axis=0)
    A_accumulator += np.sum(xi, axis=0)
    B_accumulator += np.sum(gamma[:, :, None] * (observations[None, :] == np.arange(B.shape[1])), axis=0)

    # Normalize updates by the total number of sequences
    total_sequences = len(observations_list)
    pi = pi_accumulator / total_sequences
    print(pi)
    A = A_accumulator / total_sequences
    B = (B_accumulator.T / np.sum(gamma, axis=0)).T

    return pi, A, B

In [13]:
from tqdm import tqdm

# EM algorithm
def baum_welch(observations, initialisation, n_iter=100):

    A_accumulator = np.zeros((3, 3))
    B_accumulator = np.zeros((3, 3))
    pi_accumulator = np.zeros((1, 3))

    for v in tqdm(observations):

        # Initialize parameters
        A, B, pi = initialisation()
        
        for _ in range(n_iter):
            A, B, pi = update(v, A, B, pi)
        
        # Accumulate updates
        pi_accumulator += pi
        A_accumulator += A
        B_accumulator += B

    # Normalize updates by the total number of sequences
    total_sequences = len(observations)
    pi = pi_accumulator / total_sequences
    A = A_accumulator / total_sequences
    B = (B_accumulator.T / np.sum(B_accumulator, axis=1)).T

    return pi, A, B

pi_final, A_final, B_final = baum_welch(V, initialisation=gmm_initial_parameters, n_iter=100)


100%|██████████| 500/500 [02:50<00:00,  2.92it/s]


In [14]:
pi_final, A_final, B_final

(array([[0.32400001, 0.479956  , 0.19604399]]),
 array([[0.71671304, 0.28042741, 0.00285955],
        [0.0916518 , 0.70000045, 0.20834774],
        [0.11408424, 0.15202166, 0.73389409]]),
 array([[9.44505508e-01, 5.51218481e-02, 3.72644361e-04],
        [2.50949395e-02, 9.64668287e-01, 1.02367737e-02],
        [6.09212050e-04, 6.84038636e-02, 9.30986924e-01]]))

In [15]:
# log-likelihood of data given final parameters
def log_likelihood(observations, A, B, pi):

    likelihood = 0
    for v in observations:
        likelihood += np.sum(alpha_recursion(v, A, B, pi)[-1,:])

    return np.log(likelihood / len(observations))

In [16]:
log_likelihood(V, A_final, B_final, pi_final)

-66.01397435551051

In [17]:
def posterior(observations, A, B, pi):

    alpha = alpha_recursion(observations, A, B, pi)
    beta = beta_recursion(observations, A, B)
    gamma = alpha * beta
    gamma /= np.sum(gamma, axis=1, keepdims=True)

    return gamma[-1,:]

for v in V[:10]:
    posterior_distribution = posterior(v, A_final, B_final, pi_final)
    print("Distribution: ", posterior_distribution,  "Most likley station: ", np.argmax(posterior_distribution))

Distribution:  [0.00785855 0.97021095 0.0219305 ] Most likley station:  1
Distribution:  [1.72033203e-04 3.33305335e-02 9.66497433e-01] Most likley station:  2
Distribution:  [0.01098862 0.95092158 0.0380898 ] Most likley station:  1
Distribution:  [0.84112371 0.15765625 0.00122004] Most likley station:  0
Distribution:  [0.12438917 0.87448823 0.0011226 ] Most likley station:  1
Distribution:  [9.89450084e-01 1.05453957e-02 4.52041274e-06] Most likley station:  0
Distribution:  [0.84675766 0.15207746 0.00116488] Most likley station:  0
Distribution:  [0.01099236 0.95089818 0.03810947] Most likley station:  1
Distribution:  [6.32253217e-05 2.60752285e-03 9.97329252e-01] Most likley station:  2
Distribution:  [0.0078865  0.97010013 0.02201337] Most likley station:  1
